In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8

In [ ]:
from google.colab import userdata
userdata.get('huging_face_token')

In [2]:
from unsloth import FastLanguageModel
import torch # Import PyTorch
from trl import SFTTrainer # Trainer for supervised fine-tuning (SFT)
from unsloth import is_bfloat16_supported # Checks if the hardware supports bfloat16 precision
# Hugging Face modules
from huggingface_hub import login # Lets you login to API
from transformers import TrainingArguments # Defines training hyperparameters
from datasets import load_dataset # Lets you load fine-tuning datasets
# Import weights and biases
import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
hugging_face_token = userdata.get('huging_face_token')
wand_key = userdata.get('wandb_api_key')
login(hugging_face_token)
wandb.login(key=wand_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sharadnaik001 (sharadnaik001-jj) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
run = wandb.init(
    project='Fine-tune-Qwen-2.5-0.5b-Instruct',
    job_type="training",
    anonymous="allow"
)

In [5]:
# Set parameters
max_seq_length = 2048 # Define the maximum sequence length a model can handle (i.e. how many tokens can be processed at once)
dtype = None # Set to default
load_in_4bit = True # Enables 4 bit quantization — a memory saving optimization

# Load the DeepSeek R1 model and tokenizer using unsloth — imported using: from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-0.5B-Instruct",  # Load the pre-trained DeepSeek R1 model (8B parameter version)
    max_seq_length=max_seq_length, # Ensure the model can process up to 2048 tokens at once
    dtype=dtype, # Use the default data type (e.g., FP16 or BF16 depending on hardware support)
    load_in_4bit=load_in_4bit, # Load the model in 4-bit quantization to save memory
    token=hugging_face_token, # Use hugging face token
)

==((====))==  Unsloth 2025.4.7: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [6]:
ques = ' how to reduce the body heat'

FastLanguageModel.for_inference(model)

inputs = tokenizer(ques, return_tensors="pt").to("cuda")  # Convert input to PyTorch tensor & move to GPU

output_ids=model.generate(
    input_ids = inputs.input_ids, # Tokenized input question
    attention_mask=inputs.attention_mask, # Attention mask to handle padding
    max_new_tokens=1200, # Limit response length to 1200 tokens (to prevent excessive output)
    use_cache=True, # Enable caching for faster inference
    )

# Decode the generated output tokens into human-readable text
response = tokenizer.batch_decode(output_ids)

# Extract and print only the relevant response part (after "### Response:")
print(response)

[' how to reduce the body heat loss from a room. The answer is 10°C. \\boxed{10}<|endoftext|>']


In [7]:
from unsloth.chat_templates import CHAT_TEMPLATES
print(CHAT_TEMPLATES.keys())


dict_keys(['unsloth', 'zephyr', 'chatml', 'mistral', 'llama', 'vicuna', 'vicuna_old', 'vicuna old', 'alpaca', 'gemma', 'gemma_chatml', 'gemma2', 'gemma2_chatml', 'llama-3', 'llama3', 'phi-3', 'phi-35', 'phi-3.5', 'llama-3.1', 'llama-31', 'llama-3.2', 'llama-3.3', 'llama-32', 'llama-33', 'qwen-2.5', 'qwen-25', 'qwen25', 'qwen2.5', 'phi-4', 'gemma-3', 'gemma3'])


In [8]:
from unsloth.chat_templates import get_chat_template, standardize_data_formats

tokenizer = get_chat_template(
    tokenizer,
    chat_template = 'qwen2.5'
)

In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    # task_type = "CAUSAL_LM",
    use_gradient_checkpointing="unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
    )

Unsloth 2025.4.7 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


In [28]:
import json

datas = []
with open('/content/Dataset.jsonl', 'r') as jsonl_file:
    for line in jsonl_file:
        datas.append(json.loads(line))

In [29]:

dataset = standardize_data_formats(datas)

from datasets import Dataset
# Step 3: Convert to HuggingFace Dataset
hf_dataset = Dataset.from_list(dataset)

In [13]:

# Step 4: Set up tokenizer with chat template
tokenizer = get_chat_template(tokenizer, chat_template='qwen2.5')


In [38]:

# Step 5: Apply chat template to each conversation
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples['messages'],tokenize=False)  # ← Fix here
    return {'text': texts}


# Step 6: Map and apply template
hf_dataset = hf_dataset.map(apply_chat_template, batched=True)

Map:   0%|          | 0/136 [00:00<?, ? examples/s]

In [39]:
def formatting_func(example):
    def format_single(messages):
        text = ""
        for msg in messages:
            role = msg["role"]
            content = msg["content"]
            if role == "user":
                text += "<|im_start|>user\n" + content + "<|im_end|>\n"
            elif role == "assistant":
                text += "<|im_start|>assistant\n" + content + "<|im_end|>\n"
            elif role == "system":
                text += "<|im_start|>system\n" + content + "<|im_end|>\n"
        return text

    if isinstance(example["messages"][0], list):  # Batched call
        return [format_single(msgs) for msgs in example["messages"]]
    else:  # Single example
        return format_single(example["messages"])


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = hf_dataset,
    formatting_func =  None,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)



Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/136 [00:00<?, ? examples/s]

In [41]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 136 | Num Epochs = 4 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 8,798,208/5,000,000,000 (0.18% trained)


Step,Training Loss
1,3.031900
2,3.568400
3,3.006500
4,2.436800
5,2.084100
6,2.005700
7,1.681800
8,1.432200
9,1.118800
10,1.104800


In [42]:
trainer.save_model("Qwen2.5-0.5B-finetuned")  # This creates a local folder


In [44]:
!zip -r Qwen2.5-0.5B-finetuned.zip Qwen2.5-0.5B-finetuned/


  adding: Qwen2.5-0.5B-finetuned/ (stored 0%)
  adding: Qwen2.5-0.5B-finetuned/adapter_model.safetensors (deflated 7%)
  adding: Qwen2.5-0.5B-finetuned/training_args.bin (deflated 53%)
  adding: Qwen2.5-0.5B-finetuned/added_tokens.json (deflated 67%)
  adding: Qwen2.5-0.5B-finetuned/merges.txt (deflated 57%)
  adding: Qwen2.5-0.5B-finetuned/special_tokens_map.json (deflated 69%)
  adding: Qwen2.5-0.5B-finetuned/vocab.json (deflated 61%)
  adding: Qwen2.5-0.5B-finetuned/tokenizer.json (deflated 81%)
  adding: Qwen2.5-0.5B-finetuned/tokenizer_config.json (deflated 83%)
  adding: Qwen2.5-0.5B-finetuned/adapter_config.json (deflated 56%)
  adding: Qwen2.5-0.5B-finetuned/README.md (deflated 66%)


In [52]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("Qwen2.5-0.5B-finetuned")
# model = AutoModelForCausalLM.from_pretrained("Qwen2.5-0.5B-finetuned")
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen2.5-0.5B-finetuned",
    max_seq_length = 2048,
    dtype = None,  # or "auto"
    load_in_4bit = False,  # or True if you're using quantization
)


==((====))==  Unsloth 2025.4.7: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [53]:
messages = [
    {"role": "user", "content": "hello may i the menu"}
]

# Use the tokenizer's chat template if available
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate a response
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
hello may i the menu
assistant
Sure! How many would you like to order?
